# Day 7: The Sum of Its Parts

## Parse input

In [1]:
import string
import re
from collections import OrderedDict

reinput = re.compile('Step ([A-Z]) must be finished before step ([A-Z]) can begin.')

# Parse the requirements. We assume every letter in the alphabet is a possible step.
# OrderedDict helps to execute the steps in alphabetical order as required.
requirements = OrderedDict()
for c in string.ascii_uppercase:
    requirements[c] = tuple()

# Parse requirements into a dictionary.
# The key of the dictionary will be the step to be executed,
# the value will be a tuple of steps to be executed before the step in the key can be executed.
with open('inputs/7.txt') as f:
    for l in f.readlines():
        req, step = reinput.findall(l)[0]
        requirements[step] += (req,)

## Answer #1

In [2]:
# 'state' is a dictionary with booleans indicating if a step has ran.
state = OrderedDict()
for c in requirements.keys():
    state[c] = False

answer1 = ''

# Solve it
while False in state.values():
    for step in [k for k, v in state.items() if v == False]:       
        canRun = True
        for requiredStep in requirements[step]:
            if state[requiredStep] == False:
                canRun = False
                break
        
        if canRun:
            state[step] = True
            answer1 += step
            break

print("Answer #1: %s" % answer1)

Answer #1: BITRAQVSGUWKXYHMZPOCDLJNFE


## Answer #2

In [3]:
import itertools

WORKER_COUNT = 5

# ElfWorker class represents one worker
class ElfWorker:
    def __init__(self):
        self.busyUntil = -1
        self.busyDoing = ''
        self.startTime = 0
    def startWork(self, c, tick):
        self.busyUntil = tick + 60 + string.ascii_uppercase.index(c) + 1
        self.busyDoing = c
        self.startTime = tick
    def isBusy(self, tick):
        return self.busyUntil > tick
    def isDoneOnThisTick(self, tick):
        return self.busyUntil == tick

# Reset state
state = OrderedDict()
for c in requirements.keys():
    state[c] = False
workers = [ElfWorker() for i in range(WORKER_COUNT)]
done = ''

# Start working
for tick in itertools.count():
    canRun = False
    freeWorkers = [w for w in workers if not w.isBusy(tick)]
    
    # Check for which workers have completed something this tick
    for w in workers:
        if w.isDoneOnThisTick(tick):
            done += w.busyDoing            
            state[w.busyDoing] = True
            
    # Distribute next work packages
    stepsInProgress = [w.busyDoing for w in workers if w.isBusy(tick)]
    for step in [k for k, v in state.items() if v == False and k not in stepsInProgress]:
        if (len(freeWorkers) == 0):
            break
        
        # Check if this step can be taken at this time (same as answer #1)
        canRun = True
        for requiredStep in requirements[step]:
            if state[requiredStep] == False:
                canRun = False
                break
                
        if canRun:
            # Distribute to next free worker
            freeWorker = freeWorkers.pop(0)
            freeWorker.startWork(step, tick)
    
    # Print worker status after this tick
    outLine = "%04d" % tick
    for worker in workers:
        if worker.isBusy(tick):
            outLine += " " + worker.busyDoing + (" (%04d -> %04d)" % (worker.startTime, worker.busyUntil))
        else:
            outLine += " ." + (" " * 15)
    print(outLine + " " + done)

    # Check for completion (no more steps to take and all workers free)
    if not False in state.values() and len(freeWorkers) == len(workers):
        answer2 = tick
        break

0000 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0001 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0002 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0003 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0004 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0005 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0006 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0007 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0008 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0009 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                
0010 B (0000 -> 0062) T (0000 -> 0080) V (0000 -> 0082) Y (0000 -> 0085) .                

0230 Z (0228 -> 0314) .                .                .                Q (0158 -> 0235) BTVYIWRSKMAG
0231 Z (0228 -> 0314) .                .                .                Q (0158 -> 0235) BTVYIWRSKMAG
0232 Z (0228 -> 0314) .                .                .                Q (0158 -> 0235) BTVYIWRSKMAG
0233 Z (0228 -> 0314) .                .                .                Q (0158 -> 0235) BTVYIWRSKMAG
0234 Z (0228 -> 0314) .                .                .                Q (0158 -> 0235) BTVYIWRSKMAG
0235 Z (0228 -> 0314) U (0235 -> 0316) X (0235 -> 0319) .                .                BTVYIWRSKMAGQ
0236 Z (0228 -> 0314) U (0235 -> 0316) X (0235 -> 0319) .                .                BTVYIWRSKMAGQ
0237 Z (0228 -> 0314) U (0235 -> 0316) X (0235 -> 0319) .                .                BTVYIWRSKMAGQ
0238 Z (0228 -> 0314) U (0235 -> 0316) X (0235 -> 0319) .                .                BTVYIWRSKMAGQ
0239 Z (0228 -> 0314) U (0235 -> 0316) X (0235 -> 0319) .            

0544 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0545 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0546 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0547 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0548 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0549 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0550 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0551 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0552 D (0528 -> 0592) .                .                .                .                BTVYIWRSKMAGQZUXHPOC
0

0738 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0739 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0740 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0741 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0742 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0743 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0744 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0745 F (0738 -> 0804) .                .                .                .                BTVYIWRSKMAGQZUXHPOCDLJN
0746 F (0738 -> 0804) .                .                .                .      

In [4]:
print("Answer #2: %d" % answer2)

Answer #2: 869
